In [19]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))


In [37]:
num_episodes = 5*10**4
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001
normalized_exploration_decay_rate = exploration_decay_rate * (10**4 / num_episodes)
print(normalized_exploration_decay_rate)

0.0002


In [38]:
q_table = np.zeros((state_space_size, action_space_size))
rewards_all_episodes = []
for episode in range(num_episodes):
    if episode % 1000 == 0:
        print("Training in progress, Episode number: ", episode, "expl_rate:", exploration_rate)
    # initialize new episode params
    state = env.reset()
    done = False
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
            
        # Take new action
        new_state, reward, done, info = env.step(action)
        
        # Update Q-table
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        # Set new state
        state = new_state
        
        # Add new reward
        rewards_current_episode += reward
        if done == True: 
            break
    # Exploration rate decay
    exploration_rate = min_exploration_rate + \
        ( max_exploration_rate - min_exploration_rate) * ( np.exp(-normalized_exploration_decay_rate*episode))
    # Add current episode reward to total rewards list
    rewards_all_episodes.append(rewards_current_episode)


Training in progress, Episode number:  0 expl_rate: 1
Training in progress, Episode number:  1000 expl_rate: 0.8207055704482612
Training in progress, Episode number:  2000 expl_rate: 0.6737495822176197
Training in progress, Episode number:  3000 expl_rate: 0.5534321953042277
Training in progress, Episode number:  4000 expl_rate: 0.4549246505082512
Training in progress, Episode number:  5000 expl_rate: 0.3742734941735784
Training in progress, Episode number:  6000 expl_rate: 0.3082419122110817
Training in progress, Episode number:  7000 expl_rate: 0.2541798253839962
Training in progress, Episode number:  8000 expl_rate: 0.20991753232308935
Training in progress, Episode number:  9000 expl_rate: 0.17367863179237475
Training in progress, Episode number:  10000 expl_rate: 0.14400872947014468
Training in progress, Episode number:  11000 expl_rate: 0.11971706799811506
Training in progress, Episode number:  12000 expl_rate: 0.0998287377076049
Training in progress, Episode number:  13000 expl_r

In [39]:
# Calculate and print the average reward per thousand episodes
rewards_per_thosand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000


print("********Average reward per thousand episodes********\n")
for r in rewards_per_thosand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.016000000000000007
2000 :  0.035000000000000024
3000 :  0.04900000000000004
4000 :  0.059000000000000045
5000 :  0.09000000000000007
6000 :  0.1240000000000001
7000 :  0.1540000000000001
8000 :  0.19400000000000014
9000 :  0.25000000000000017
10000 :  0.2770000000000002
11000 :  0.3760000000000003
12000 :  0.3850000000000003
13000 :  0.3920000000000003
14000 :  0.46300000000000036
15000 :  0.47100000000000036
16000 :  0.5160000000000003
17000 :  0.5060000000000003
18000 :  0.5810000000000004
19000 :  0.5980000000000004
20000 :  0.6080000000000004
21000 :  0.6090000000000004
22000 :  0.6520000000000005
23000 :  0.6330000000000005
24000 :  0.6620000000000005
25000 :  0.6410000000000005
26000 :  0.6700000000000005
27000 :  0.6450000000000005
28000 :  0.6900000000000005
29000 :  0.6530000000000005
30000 :  0.6490000000000005
31000 :  0.6750000000000005
32000 :  0.6870000000000005
33000 :  0.6790000000000005
34000 :  0.63800000

In [40]:
# Pring updated Q-table
print("\n\n***Q-table***\n")
print(q_table)



***Q-table***

[[0.57605106 0.51095667 0.51189128 0.50949747]
 [0.37258509 0.37231067 0.30798245 0.53783669]
 [0.40643167 0.40229146 0.41236671 0.49551838]
 [0.29430262 0.3622298  0.27993841 0.45710801]
 [0.5959662  0.44448833 0.40703459 0.293807  ]
 [0.         0.         0.         0.        ]
 [0.19508925 0.11227317 0.33377066 0.0701896 ]
 [0.         0.         0.         0.        ]
 [0.38809834 0.48066923 0.34580412 0.62122122]
 [0.51974455 0.65662733 0.45149114 0.48705704]
 [0.57520924 0.40845545 0.18024615 0.32272873]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.54993035 0.39794883 0.77779798 0.52321788]
 [0.75969328 0.90411625 0.76603969 0.7299915 ]
 [0.         0.         0.         0.        ]]


In [45]:
# Watch our agent play Frozen Lake by playing the best action
# from each state according to the Q-table

for episode in range(10):
    # initialize new episdoe params
    state = env.reset()
    done = False
    print("****EPISODE ", episode+1, "****\n\n\n")
    time.sleep(1)
    for step in range(max_steps_per_episode):
        # Show current state of environment on screen
        clear_output(wait=True)
        env.render()
        time.sleep(0.2)
        # Choose action with highest Q-value for current state
        action = np.argmax(q_table[state,:])
        # Take new action
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                #Agent reached the goal and won episode
                print("****You reached the goal!****")
                time.sleep(2)
            else:
                # Agent lost
                print("****You fell through the hole!****")
                time.sleep(2)
                clear_output(wait=True)
        # set new state
            break
        state = new_state
env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG
****You reached the goal!****
